In [76]:
import pandas as pd

import numpy as np

In [5]:
df = pd.read_csv('PartD_Prescriber_PUF_NPI_14.txt', delimiter='\t')

df.columns

Index([u'NPI', u'NPPES_PROVIDER_LAST_ORG_NAME', u'NPPES_PROVIDER_FIRST_NAME',
       u'NPPES_PROVIDER_MI', u'NPPES_CREDENTIALS', u'NPPES_PROVIDER_GENDER',
       u'NPPES_ENTITY_CODE', u'NPPES_PROVIDER_STREET1',
       u'NPPES_PROVIDER_STREET2', u'NPPES_PROVIDER_CITY',
       u'NPPES_PROVIDER_ZIP5', u'NPPES_PROVIDER_ZIP4', u'NPPES_PROVIDER_STATE',
       u'NPPES_PROVIDER_COUNTRY', u'SPECIALTY_DESCRIPTION',
       u'DESCRIPTION_FLAG', u'MEDICARE_PRVDR_ENROLL_STATUS', u'BENE_COUNT',
       u'TOTAL_CLAIM_COUNT', u'TOTAL_DRUG_COST', u'TOTAL_DAY_SUPPLY',
       u'BENE_COUNT_GE65', u'BENE_COUNT_GE65_SUPPRESS_FLAG',
       u'TOTAL_CLAIM_COUNT_GE65', u'GE65_SUPPRESS_FLAG',
       u'TOTAL_DRUG_COST_GE65', u'TOTAL_DAY_SUPPLY_GE65', u'BRAND_CLAIM_COUNT',
       u'BRAND_SUPPRESS_FLAG', u'BRAND_DRUG_COST', u'GENERIC_CLAIM_COUNT',
       u'GENERIC_SUPPRESS_FLAG', u'GENERIC_DRUG_COST', u'OTHER_CLAIM_COUNT',
       u'OTHER_SUPPRESS_FLAG', u'OTHER_DRUG_COST', u'MAPD_CLAIM_COUNT',
       u'MAPD_SUPPRESS_

In [10]:
# Question 1
df.BENE_COUNT.dropna().mean()

148.4730131955417

In [26]:
# Question 2

avg_claim_length = df['TOTAL_DAY_SUPPLY'].astype('float') / df['TOTAL_CLAIM_COUNT']
print avg_claim_length.dropna().median()


29.1037983864


In [46]:
# Question 3
table = df[['SPECIALTY_DESCRIPTION', 'TOTAL_CLAIM_COUNT', 'BRAND_CLAIM_COUNT','BRAND_SUPPRESS_FLAG']]
table = table[table['BRAND_CLAIM_COUNT'].notnull()]
specialties = table.groupby('SPECIALTY_DESCRIPTION').aggregate(sum)
specialties = specialties[specialties['TOTAL_CLAIM_COUNT'] >= 1000]
print len(specialties)
print specialties.head()

123
                       TOTAL_CLAIM_COUNT  BRAND_CLAIM_COUNT
SPECIALTY_DESCRIPTION                                      
Acupuncturist                       3852              884.0
Addiction Medicine                101356            25173.0
Allergy/Immunology               2034665           811506.0
Anesthesiology                   3749631           577494.0
Behavioral Analyst                  2464              561.0


In [47]:
brand_fraction = specialties['BRAND_CLAIM_COUNT'] / specialties['TOTAL_CLAIM_COUNT']
print brand_fraction.std()

0.0880682412859


In [61]:
# Question 4
table = df[['NPPES_PROVIDER_STATE', 'OPIOID_BENE_COUNT', 'ANTIBIOTIC_BENE_COUNT']]
table = table[table['OPIOID_BENE_COUNT'].notnull()]
table = table[table['ANTIBIOTIC_BENE_COUNT'].notnull()]
table2 = table.groupby('NPPES_PROVIDER_STATE').aggregate(sum)
OP2AB_RATIO = table2['OPIOID_BENE_COUNT'] / table2['ANTIBIOTIC_BENE_COUNT']
print OP2AB_RATIO.head()
print OP2AB_RATIO.max(), OP2AB_RATIO.min()
print OP2AB_RATIO.max() - OP2AB_RATIO.min()


NPPES_PROVIDER_STATE
AA    0.564286
AE    0.805024
AK    0.917580
AL    0.852258
AP    0.807203
dtype: float64
1.47058823529 0.404761904762
1.06582633053


In [67]:
# Question 5
table = df[['TOTAL_CLAIM_COUNT', 'TOTAL_CLAIM_COUNT_GE65', 'LIS_CLAIM_COUNT']].dropna(0)

ge65_frac = table['TOTAL_CLAIM_COUNT_GE65'] / table['TOTAL_CLAIM_COUNT']
lis_frac = table['LIS_CLAIM_COUNT'] / table['TOTAL_CLAIM_COUNT']

print ge65_frac.corr(lis_frac,method='pearson')

-0.663524876752


In [114]:
# Question 6
table = df[['NPPES_PROVIDER_STATE', 'SPECIALTY_DESCRIPTION', 'OPIOID_DAY_SUPPLY', 'OPIOID_CLAIM_COUNT']].dropna(0)
#print len(table)
#print table.head()

table['avg_len'] = table['OPIOID_DAY_SUPPLY'] / table['OPIOID_CLAIM_COUNT']
table = table.drop('OPIOID_DAY_SUPPLY',1).drop('OPIOID_CLAIM_COUNT',1)
op_by_state_spec = table.groupby(('NPPES_PROVIDER_STATE', 'SPECIALTY_DESCRIPTION'))
state_spec_count = op_by_state_spec.size()
print state_spec_count.head()

NPPES_PROVIDER_STATE  SPECIALTY_DESCRIPTION
AA                    Dentist                  2
                      Diagnostic Radiology     1
                      Family Practice          1
                      General Practice         1
                      General Surgery          1
dtype: int64


In [116]:
state_spec_avg = op_by_state_spec.aggregate(np.mean)
print state_spec_avg.columns
state_spec_avg = state_spec_avg[state_spec_count >= 100].reset_index()
print len(state_spec_avg)
print state_spec_avg.head()

Index([u'avg_len'], dtype='object')
1079
  NPPES_PROVIDER_STATE SPECIALTY_DESCRIPTION    avg_len
0                   AK       Family Practice  21.893003
1                   AK     Internal Medicine  20.940059
2                   AK    Nurse Practitioner  21.692129
3                   AK   Physician Assistant  16.446126
4                   AL        Anesthesiology  24.443542


In [128]:
# q6: across all states
op_by_spec = table[['SPECIALTY_DESCRIPTION', 'avg_len']].groupby('SPECIALTY_DESCRIPTION',as_index=False).aggregate(np.mean)
state_spec_with_nat = state_spec_avg.merge(op_by_spec,
                     how='left',
                     right_on='SPECIALTY_DESCRIPTION',
                     left_on='SPECIALTY_DESCRIPTION',
                     suffixes=['','_allstates'])
state_spec_with_nat['normalized_ratio'] = state_spec_with_nat['avg_len'] / state_spec_with_nat['avg_len_allstates']
#print state_spec_with_nat.head()
print state_spec_with_nat['normalized_ratio'].max()


2.04039908783


In [129]:
# 2013 data
df13 = pd.read_csv('PartD_Prescriber_PUF_NPI_13.tab', delimiter='\t')

df13.columns

C:\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index([u'NPI', u'NPPES_PROVIDER_LAST_ORG_NAME', u'NPPES_PROVIDER_FIRST_NAME',
       u'NPPES_PROVIDER_MI', u'NPPES_CREDENTIALS', u'NPPES_PROVIDER_GENDER',
       u'NPPES_ENTITY_CODE', u'NPPES_PROVIDER_STREET1',
       u'NPPES_PROVIDER_STREET2', u'NPPES_PROVIDER_CITY',
       u'NPPES_PROVIDER_ZIP', u'NPPES_PROVIDER_STATE',
       u'NPPES_PROVIDER_COUNTRY', u'SPECIALTY_DESC', u'DESCRIPTION_FLAG',
       u'BENE_COUNT', u'TOTAL_CLAIM_COUNT', u'TOTAL_DRUG_COST',
       u'TOTAL_DAY_SUPPLY', u'BENE_COUNT_GE65', u'BENE_COUNT_GE65_REDACT_FLAG',
       u'TOTAL_CLAIM_COUNT_GE65', u'GE65_REDACT_FLAG', u'TOTAL_DRUG_COST_GE65',
       u'TOTAL_DAY_SUPPLY_GE65', u'BRAND_CLAIM_COUNT', u'BRAND_REDACT_FLAG',
       u'BRAND_CLAIM_COST', u'GENERIC_CLAIM_COUNT', u'GENERIC_REDACT_FLAG',
       u'GENERIC_CLAIM_COST', u'OTHER_CLAIM_COUNT', u'OTHER_REDACT_FLAG',
       u'OTHER_CLAIM_COST', u'MAPD_CLAIM_COUNT', u'MAPD_REDACT_FLAG',
       u'MAPD_CLAIM_COST', u'PDP_CLAIM_COUNT', u'PDP_REDACT_FLAG',
       u'PDP_C

In [143]:
# Question 7

table13 = df13[['NPI', 'TOTAL_DRUG_COST', 'TOTAL_DAY_SUPPLY']].dropna(1)
#print table13.head()
table13['cost_per_day_13'] = table13['TOTAL_DRUG_COST'] / table13['TOTAL_DAY_SUPPLY']
#print table13.drop('TOTAL_DRUG_COST',1).drop('TOTAL_DAY_SUPPLY',1).head()
table13.drop(['TOTAL_DRUG_COST', 'TOTAL_DAY_SUPPLY'],1,inplace=True)

table14 = df[['NPI', 'TOTAL_DRUG_COST', 'TOTAL_DAY_SUPPLY']].dropna(1)
table14['cost_per_day_14'] = table14['TOTAL_DRUG_COST'] / table14['TOTAL_DAY_SUPPLY']
table14.drop(['TOTAL_DRUG_COST', 'TOTAL_DAY_SUPPLY'],1,inplace=True)

combined = table13.merge(table14,left_on='NPI', right_on='NPI', how='inner')
combined['inflation'] = combined['cost_per_day_14'] / combined['cost_per_day_13']
print len(combined)
print combined.head()
print combined.inflation.mean()
print combined.inflation.median()

970492
          NPI  cost_per_day_13  cost_per_day_14  inflation
0  1518048750         2.224526         3.365580   1.512942
1  1952310666         3.016262         3.672300   1.217500
2  1467482471         2.115574         0.606025   0.286459
3  1801093968         1.877858         2.472467   1.316642
4  1588763981         1.077026         1.111436   1.031949
1.24451835244
1.02428470098


In [145]:
# Question 8
table13 = df13[['NPI', 'SPECIALTY_DESC']].dropna(1)
table14 = df[['NPI', 'SPECIALTY_DESCRIPTION']].dropna(1)
specialties = table13.merge(table14,how='inner',left_on='NPI',right_on='NPI')
print len(specialties)
print specialties.head()

970492
          NPI      SPECIALTY_DESC SPECIALTY_DESCRIPTION
0  1518048750  Pediatric Medicine    Pediatric Medicine
1  1952310666          Psychiatry            Psychiatry
2  1467482471             Dentist               Dentist
3  1801093968  Emergency Medicine    Emergency Medicine
4  1588763981             Urology               Urology


In [161]:
specialties['changed'] = specialties['SPECIALTY_DESC'] != specialties['SPECIALTY_DESCRIPTION']
nChanged = specialties[['SPECIALTY_DESC', 'changed']].groupby('SPECIALTY_DESC').aggregate(sum)
count = specialties.groupby('SPECIALTY_DESC').size()

ans = nChanged[count >= 1000].changed / count[count >= 1000]

print ans.head()
print ans.nlargest(5)
print ans.max()

SPECIALTY_DESC
Allergy/Immunology           0.019603
Anesthesiology               0.044754
Cardiac Electrophysiology    0.048411
Cardiac Surgery              0.088031
Cardiology                   0.035582
dtype: float64
SPECIALTY_DESC
Student in an Organized Health Care Education/Training Program    0.275951
Orthopaedic Surgery                                               0.208354
Psychiatry & Neurology                                            0.156195
Neuropsychiatry                                                   0.146896
Geriatric Medicine                                                0.127205
dtype: float64
0.275951086957


In [162]:
print ans['Orthopaedic Surgery']

0.208353569694
